<a href="https://colab.research.google.com/github/sanazy/Plate-Detection-Project/blob/master/06_inference_frozen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
TEST_PATH = '/content/drive/My Drive/Traffic/test/'
EFF_PATH = '/content/drive/My\ Drive/Traffic/EfficientDet'
%cd {EFF_PATH}

/content/drive/My Drive/Traffic/EfficientDet


In [ ]:
!pip install -r {EFF_PATH}/requirements.txt -q

In [ ]:
import tensorflow as tf
import numpy as np
import cv2
import os
import time
import glob
from utils import preprocess_image
from tensorflow.python.platform import gfile

from utils.anchors import anchors_for_shape
from utils.draw_boxes import draw_boxes
from utils import postprocess_boxes
from google.colab.patches import cv2_imshow

In [ ]:
def get_frozen_graph(graph_file):
  with tf.gfile.FastGFile(graph_file, "rb") as f:
    graph_def = tf.compat.v1.GraphDef()
    graph_def.ParseFromString(f.read())
  return graph_def

In [ ]:
def inference_frozen():
  phi = 0
  model_path = 'checkpoints/2020-06-21.pb'
  image_sizes = (256, 640, 768, 896, 1024, 1280, 1408)
  image_size = image_sizes[phi]
  classes = ['light', 'heavy', 'unknown']
  num_classes = len(classes)
  score_threshold = 0.5
  colors = [np.random.randint(0, 256, 3).tolist() for i in range(num_classes)]

  output_names = {
    'output_boxes': 'filtered_detections/map/TensorArrayStack/TensorArrayGatherV3:0',
    'output_scores': 'filtered_detections/map/TensorArrayStack_1/TensorArrayGatherV3:0',
    'output_labels': 'filtered_detections/map/TensorArrayStack_2/TensorArrayGatherV3:0'
  }

  graph = tf.Graph()
  graph.as_default()
  sess = tf.Session()
  graph = get_frozen_graph(model_path)
  tf.import_graph_def(graph, name='')

  output_boxes = sess.graph.get_tensor_by_name(output_names['output_boxes'])
  output_scores = sess.graph.get_tensor_by_name(output_names['output_scores'])
  output_labels = sess.graph.get_tensor_by_name(output_names['output_labels'])
  
  #image_path = '../test/2020-02-27_10-30-14_622151943_47_1_2_L1_I.jpg'
  for image_path in glob.glob(TEST_PATH + '*.jpg'):
    image = cv2.imread(image_path)
    src_image = image.copy()
    image = image[:, :, ::-1]
    h, w = image.shape[:2]
    
    image, scale = preprocess_image(image, image_size=image_size)
    anchors = anchors_for_shape((image_size, image_size))
    
    # run network
    start = time.time()
    image_batch = np.expand_dims(image, axis=0)
    anchors_batch = np.expand_dims(anchors, axis=0)
    feed_dict = {"input_1:0": image_batch }#, "input_4:0": anchors_batch}
    boxes, scores, labels = sess.run([output_boxes, output_scores, output_labels], feed_dict)

    boxes, scores, labels = np.squeeze(boxes), np.squeeze(scores), np.squeeze(labels)
    print(time.time() - start)
    boxes = postprocess_boxes(boxes=boxes,
                              scale=scale,
                              height=h,
                              width=w)

    # select indices which have a score above the threshold
    indices = np.where(scores[:] > score_threshold)[0]

    # select those detections
    boxes = boxes[indices]
    labels = labels[indices]

    draw_boxes(src_image, boxes, scores, labels, colors, classes)

    #cv2_imshow(src_image)

In [ ]:
inference_frozen()

2.5286498069763184
0.14187049865722656
0.14195513725280762
0.14552664756774902
0.14848709106445312
0.15582275390625
0.14661359786987305
0.1461784839630127
0.14056181907653809
0.14246392250061035
0.1468520164489746
0.1483912467956543
0.14765048027038574
0.14009404182434082
0.1398756504058838
0.14130377769470215
0.1423187255859375
0.1410677433013916
0.13984012603759766
0.14091229438781738
0.13939857482910156
0.1402435302734375
0.14121198654174805
0.14084219932556152
0.13985633850097656
0.14052319526672363
0.1419074535369873
0.14150047302246094
0.14041852951049805
0.1397538185119629
0.14284086227416992
0.13975286483764648
0.14168691635131836
0.14147329330444336
0.14380931854248047
0.1414322853088379
0.14638614654541016
0.14049196243286133
0.13968181610107422
0.14676451683044434
0.1407756805419922
0.14010167121887207
0.14790749549865723
0.14920520782470703
0.14244723320007324
0.14452099800109863
0.140120267868042
0.14032411575317383
0.13973617553710938
0.1414811611175537
0.1407923698425293